In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline


In [3]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [4]:
import pandas as pd
data = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/train_data_add_gid_weather (1).csv')
data = data.drop(columns=['기온 QC플래그', '강수량(mm)',
       '강수량 QC플래그', '풍속(m/s)', '풍속 QC플래그', '풍향(16방위)', '풍향 QC플래그', '습도(%)',
       '습도 QC플래그', '증기압(hPa)', '이슬점온도(°C)', '현지기압(hPa)', '현지기압 QC플래그',
       '해면기압(hPa)', '해면기압 QC플래그', '일조(hr)', '일조 QC플래그', '일사(MJ/m2)',
       '일사 QC플래그', '적설(cm)', '3시간신적설(cm)', '전운량(10분위)', '중하층운량(10분위)',
       '운형(운형약어)', '최저운고(100m )', '시정(10m)', '지면상태(지면상태코드)', '현상번호(국내식)',
       '지면온도(°C)', '지면온도 QC플래그', '5cm 지중온도(°C)', '10cm 지중온도(°C)',
       '20cm 지중온도(°C)', '30cm 지중온도(°C)'])

In [5]:
data

,gid,DATE,TIME,RIDE_DEMAND,ALIGHT_DEMAND,week,holiday,DATETIME,지점,지점명,기온(°C),total_population,total_building,old_population,working_population,building_to_land_ratio,floor_area_ratio,bus_count
0,다마9599,2023-06-01,5,1,0,Thursday,0,2023-06-01 05:00,133,대전,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,다마9599,2023-06-01,6,2,5,Thursday,0,2023-06-01 06:00,133,대전,18.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,다마9599,2023-06-01,7,2,7,Thursday,0,2023-06-01 07:00,133,대전,18.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,다마9599,2023-06-01,8,3,12,Thursday,0,2023-06-01 08:00,133,대전,20.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,다마9599,2023-06-01,9,2,31,Thursday,0,2023-06-01 09:00,133,대전,22.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539118,라바0421,2023-08-24,21,0,1,Thursday,0,2023-08-24 21:00,133,대전,24.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0
539119,라바0421,2023-08-24,22,0,0,Thursday,0,2023-08-24 22:00,133,대전,24.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0
539120,라바0421,2023-08-24,23,0,0,Thursday,0,2023-08-24 23:00,133,대전,24.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0
539121,라바0421,2023-08-24,0,0,0,Thursday,0,2023-08-24 00:00,133,대전,23.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(data, test_size=0.2, ### [] 안 기준 (파일에 따라 수정 가능)
                                       random_state = 42)

In [7]:
train = train_set.drop("RIDE_DEMAND", axis=1)   ## 트레인 변수
data_labels =  train_set["RIDE_DEMAND"].copy()  ## 타겟 변수
train.shape

(431298, 17)

In [8]:
test = test_set.drop("RIDE_DEMAND", axis=1)   ## 트레인 변수
test_labels =  test_set["RIDE_DEMAND"].copy()  ## 타겟 변수
test.shape


(107825, 17)

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split


train_set, test_set = train_test_split(data, test_size=0.2, random_state=42)


In [10]:
!pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.0/224.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.7/285.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.3/372.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 88.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 7.0 MB/s e

In [ ]:
pip install fastapi kaleido python-multipart uvicorn
pip install --upgrade setuptools
pip install --upgrade requests
pip install --upgrade protobuf
pip install --upgrade torch
pip install --upgrade yfinance

In [11]:
import pandas as pd
# 저희는 Tabular Data를 다루기 때문에 아래 라이브러리를 호출합니다.
from autogluon.tabular import TabularDataset, TabularPredictor

In [18]:
selected_features = ['gid','DATE','TIME','RIDE_DEMAND',	"ALIGHT_DEMAND"	,'week','holiday','bus_count',
                     'floor_area_ratio','old_population','total_population','working_population','total_building']  # 실제 사용하는 특성 이름으로 대체

selected_features1 = ['gid','DATE','TIME',	"ALIGHT_DEMAND"	,'week','holiday','bus_count',
                     'floor_area_ratio','old_population','total_population','working_population',
                      'total_building','DATETIME', '기온(°C)', 'building_to_land_ratio']  # 실제 사용하는 특성 이름으로 대체


train_set_selected = train_set[selected_features]
test_set_selected = test_set[selected_features1]


In [27]:
test_set_selected.columns


Index(['gid', 'DATE', 'TIME', 'ALIGHT_DEMAND', 'week', 'holiday', 'bus_count',
       'floor_area_ratio', 'old_population', 'total_population',
       'working_population', 'total_building', 'DATETIME', '기온(°C)',
       'building_to_land_ratio'],
      dtype='object')

In [13]:
predictor = TabularPredictor(label='RIDE_DEMAND', eval_metric='mae').fit(train_set)

No path specified. Models will be saved in: "AutogluonModels/ag-20231122_153440/"
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20231122_153440/"
AutoGluon Version:  0.8.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Wed Aug 30 11:19:59 UTC 2023
Disk Space Avail:   204.31 GB / 242.49 GB (84.3%)
Train Data Rows:    431298
Train Data Columns: 17
Label Column: RIDE_DEMAND
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == int and many unique label-values observed).
	Label info (max, min, mean, stddev): (2485, 0, 39.10976, 106.50733)
	If 'regression' is not the correct problem_type, please manually specify the problem_type par

[1000]	valid_set's l1: 4.77207
[2000]	valid_set's l1: 4.44102
[3000]	valid_set's l1: 4.29843
[4000]	valid_set's l1: 4.19686
[5000]	valid_set's l1: 4.15042
[6000]	valid_set's l1: 4.11495
[7000]	valid_set's l1: 4.08237
[8000]	valid_set's l1: 4.06003
[9000]	valid_set's l1: 4.04876
[10000]	valid_set's l1: 4.03608


	-4.0354	 = Validation score   (-mean_absolute_error)
	63.88s	 = Training   runtime
	0.27s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's l1: 4.46551
[2000]	valid_set's l1: 4.19157
[3000]	valid_set's l1: 4.07833
[4000]	valid_set's l1: 4.02938
[5000]	valid_set's l1: 3.99096
[6000]	valid_set's l1: 3.97723
[7000]	valid_set's l1: 3.96083
[8000]	valid_set's l1: 3.96109
[9000]	valid_set's l1: 3.95741
[10000]	valid_set's l1: 3.94178


	-3.9413	 = Validation score   (-mean_absolute_error)
	55.88s	 = Training   runtime
	0.26s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-4.3784	 = Validation score   (-mean_absolute_error)
	51.18s	 = Training   runtime
	0.13s	 = Validation runtime
Fitting model: CatBoost ...
	-4.5041	 = Validation score   (-mean_absolute_error)
	629.59s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: ExtraTreesMSE ...
	-4.3871	 = Validation score   (-mean_absolute_error)
	21.38s	 = Training   runtime
	0.14s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	-5.251	 = Validation score   (-mean_absolute_error)
	348.47s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: XGBoost ...
	-4.3108	 = Validation score   (-mean_absolute_error)
	93.18s	 = Training   runtime
	0.37s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	-6.6874	 = Validation score   (-mean_absolute_error)
	662.58s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: 

[1000]	valid_set's l1: 4.12305
[2000]	valid_set's l1: 3.95228
[3000]	valid_set's l1: 3.88766
[4000]	valid_set's l1: 3.85357
[5000]	valid_set's l1: 3.83073
[6000]	valid_set's l1: 3.8194
[7000]	valid_set's l1: 3.80538
[8000]	valid_set's l1: 3.80682
[9000]	valid_set's l1: 3.80234
[10000]	valid_set's l1: 3.80099


	-3.7995	 = Validation score   (-mean_absolute_error)
	113.69s	 = Training   runtime
	0.33s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	-3.7679	 = Validation score   (-mean_absolute_error)
	0.39s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 2057.24s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20231122_153440/")


In [147]:
predictions = predictor.predict(test_set_selected)

In [148]:
test_labels = test_set[['RIDE_DEMAND']]
test_labels

,RIDE_DEMAND
409604,0
222701,2
452920,0
80342,3
331392,152
...,...
41158,2
248767,0
185993,0
234493,0


In [149]:
predictions

409604      0.166348
222701      1.480287
452920      0.280611
80342       3.203495
331392    139.685776
             ...    
41158       0.857157
248767      0.284840
185993      1.396253
234493      0.579749
57303       6.075199
Name: RIDE_DEMAND, Length: 107825, dtype: float32

In [150]:
import numpy as np

# 음수를 0으로 바꾸기
predictions[predictions < 0] = 0


In [151]:
predictions

409604      0.166348
222701      1.480287
452920      0.280611
80342       3.203495
331392    139.685776
             ...    
41158       0.857157
248767      0.284840
185993      1.396253
234493      0.579749
57303       6.075199
Name: RIDE_DEMAND, Length: 107825, dtype: float32

In [153]:
from sklearn.metrics import mean_absolute_error
mae_score = mean_absolute_error(test_labels, predictions)
mae_score

3.934952216447344

In [154]:
import pandas as pd

# 예측값 반올림
rounded_predictions = np.round(predictions)

In [155]:
rounded_predictions

409604      0.0
222701      1.0
452920      0.0
80342       3.0
331392    140.0
          ...  
41158       1.0
248767      0.0
185993      1.0
234493      1.0
57303       6.0
Name: RIDE_DEMAND, Length: 107825, dtype: float32

In [156]:
from sklearn.metrics import mean_absolute_error
mae_score = mean_absolute_error(test_labels, rounded_predictions)
mae_score

3.8972223510317643

In [99]:
import joblib
# 모델 객체를 직렬화하여 .pkl 파일로 저장
joblib.dump(predictor, 'my_model.pickle')

# 저장된 파일 불러와서 확인
loaded_model = joblib.load('my_model.pickle')
print(loaded_model)

# 완성된 모델로 정답 도출하기

In [126]:
test_re = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/test1.csv')

In [127]:
test_re

,gid,DATE,TIME,ALIGHT_DEMAND,week,holiday,bus_count,floor_area_ratio,old_population,total_population,working_population,total_building,DATETIME,기온(°C),building_to_land_ratio
0,다바8116,2023-06-24,5,0,Saturday,1,6,108.16,138.0,1051.0,875.0,208.0,2023-06-24 05:00,22.4,36.29
1,다바8116,2023-06-24,6,15,Saturday,1,6,108.16,138.0,1051.0,875.0,208.0,2023-06-24 06:00,22.4,36.29
2,다바8116,2023-06-24,7,22,Saturday,1,6,108.16,138.0,1051.0,875.0,208.0,2023-06-24 07:00,23.3,36.29
3,다바8116,2023-06-24,8,63,Saturday,1,6,108.16,138.0,1051.0,875.0,208.0,2023-06-24 08:00,25.2,36.29
4,다바8116,2023-06-24,9,65,Saturday,1,6,108.16,138.0,1051.0,875.0,208.0,2023-06-24 09:00,26.6,36.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85458,다바9830,2023-08-30,22,0,Wednesday,0,1,35.89,NaN,NaN,NaN,4.0,2023-08-30 22:00,20.0,25.80
85459,다바9830,2023-08-30,23,0,Wednesday,0,1,35.89,NaN,NaN,NaN,4.0,2023-08-30 23:00,20.1,25.80
85460,다바9830,2023-08-30,0,0,Wednesday,0,1,35.89,NaN,NaN,NaN,4.0,2023-08-30 00:00,21.6,25.80
85461,다바9830,2023-08-30,1,0,Wednesday,0,1,35.89,NaN,NaN,NaN,4.0,2023-08-30 01:00,21.6,25.80


In [128]:
predictions = loaded_model.predict(test_re)

In [132]:
# 예측값 음수를 0으로 바꾸기
predictions[predictions < 0] = 0

# 예측값 소숫점 반올림
import pandas as pd
rounded_predictions = np.round(predictions)

In [135]:
# 예측값을 'RIDE_DEMAND' 칼럼으로 하는 데이터프레임 생성
result_df = pd.DataFrame({'RIDE_DEMAND': rounded_predictions})

In [136]:
result_df

,RIDE_DEMAND
0,4.0
1,5.0
2,12.0
3,23.0
4,28.0
...,...
85458,0.0
85459,0.0
85460,0.0
85461,0.0


In [137]:
# 원래 데이터프레임에 'RIDE_DEMAND' 칼럼 추가
test_re['RIDE_DEMAND'] = result_df['RIDE_DEMAND']

In [138]:
test_re.columns

Index(['gid', 'DATE', 'TIME', 'ALIGHT_DEMAND', 'week', 'holiday', 'bus_count',
       'floor_area_ratio', 'old_population', 'total_population',
       'working_population', 'total_building', 'DATETIME', '기온(°C)',
       'building_to_land_ratio', 'RIDE_DEMAND'],
      dtype='object')

In [139]:
test_re1 = test_re[['gid', 'DATE', 'TIME', 'ALIGHT_DEMAND','RIDE_DEMAND']]

In [140]:
test_re1

,gid,DATE,TIME,ALIGHT_DEMAND,RIDE_DEMAND
0,다바8116,2023-06-24,5,0,4.0
1,다바8116,2023-06-24,6,15,5.0
2,다바8116,2023-06-24,7,22,12.0
3,다바8116,2023-06-24,8,63,23.0
4,다바8116,2023-06-24,9,65,28.0
...,...,...,...,...,...
85458,다바9830,2023-08-30,22,0,0.0
85459,다바9830,2023-08-30,23,0,0.0
85460,다바9830,2023-08-30,0,0,0.0
85461,다바9830,2023-08-30,1,0,0.0


In [143]:
test_re1.to_csv('test_final.csv', index=False)